In [1]:

import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

#Assigning dotenv file variabe's values  to the other variables for access
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")
user  = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")

# Establishing Connection
con = psycopg2.connect(
    host = host,
    port = port,
    database = database,
    user = user,
    password = password
)
cur = con.cursor()
print("✅Connection Successful")


✅Connection Successful


📌Creating Dictionary for 4 csv files

In [3]:
csv_map = { 
    "../Preprocessed_Datasets/cleaned_providers_data.csv": {
       "Table": "Providers",
       "Schema": '''
           CREATE TABLE IF NOT EXISTS Providers (
               Provider_ID INTEGER PRIMARY KEY,
               Name TEXT,
               Type TEXT,
               Address TEXT,
               City TEXT,
               Contact TEXT
           );
       ''',
       "Columns": ['Provider_ID', 'Name', 'Type', 'Address', 'City', 'Contact']
    },

    "../Preprocessed_Datasets/cleaned_receivers_data.csv": {
       "Table": "Receivers",
       "Schema": '''
           CREATE TABLE IF NOT EXISTS Receivers (
               Receiver_ID INTEGER PRIMARY KEY,
               Name TEXT,
               Type TEXT,
               City TEXT,
               Contact TEXT
           );
       ''',
       "Columns": ['Receiver_ID', 'Name', 'Type', 'City', 'Contact']
    },

    "../Preprocessed_Datasets/cleaned_food_listings_data.csv": {
       "Table": "Food_Listings",
       "Schema": '''
           CREATE TABLE IF NOT EXISTS Food_Listings (
               Food_ID INTEGER PRIMARY KEY,
               Food_Name TEXT,
               Quantity INTEGER,
               Expiry_Date DATE,
               Provider_ID INTEGER,
               Provider_Type TEXT,
               Location TEXT,
               Food_Type TEXT,
               Meal_Type TEXT,
               FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
           );
       ''',
       "Columns": ['Food_ID', 'Food_Name', 'Quantity', 'Expiry_Date', 'Provider_ID', 'Provider_Type', 'Location', 'Food_Type', 'Meal_Type']
    },

    "../Preprocessed_Datasets/cleaned_claims_data.csv": {
       "Table": "Claims",
       "Schema": '''
           CREATE TABLE IF NOT EXISTS Claims (
               Claim_ID INTEGER PRIMARY KEY,
               Food_ID INTEGER,
               Receiver_ID INTEGER,
               Status TEXT,
               Timestamp TIMESTAMP,
               FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID),
               FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
           );
       ''',
       "Columns": ['Claim_ID', 'Food_ID', 'Receiver_ID', 'Status', 'Timestamp']
    }
}



📌Loading and creating tables in the database for csv files data

In [4]:
import pandas as pd
# ✅ Insert loop with logging and row-level error handling
skipped_rows = []

for file, details in csv_map.items():
    print(f"\n📁 Loading file: {file}")
    try:
        df = pd.read_csv(file)
    except Exception as e:
        print(f"❌ Error reading {file}: {e}")
        continue

    print(f"🧱 Creating table: {details['Table']}")
    cur.execute(details["Schema"])
    con.commit()

    placeholders = ", ".join(["%s"] * len(details["Columns"]))
    columns = ", ".join(details["Columns"])
    insert_query = f"INSERT INTO {details['Table']} ({columns}) VALUES ({placeholders})"

    for _, row in df.iterrows():
        try:
            cur.execute(insert_query, tuple(row[col] for col in details["Columns"]))
            con.commit()
        except Exception as e:
            con.rollback()
            print(f"⚠️ Skipped row due to error: {e}")
            skipped_rows.append(row)

    print(f"✅ Finished inserting: {details['Table']}")

# ✅ Save skipped rows (if any)
if skipped_rows:
    skipped_df = pd.DataFrame(skipped_rows)
    skipped_df.to_csv("skipped_rows.csv", index=False)
    print("📄 Skipped rows saved to 'skipped_rocws.csv'")

# ✅ Cleanup
cur.close()
con.close()
print("\n🎉 All CSV files inserted successfully!")


📁 Loading file: ../Preprocessed_Datasets/cleaned_providers_data.csv
🧱 Creating table: Providers
✅ Finished inserting: Providers

📁 Loading file: ../Preprocessed_Datasets/cleaned_receivers_data.csv
🧱 Creating table: Receivers
✅ Finished inserting: Receivers

📁 Loading file: ../Preprocessed_Datasets/cleaned_food_listings_data.csv
🧱 Creating table: Food_Listings
✅ Finished inserting: Food_Listings

📁 Loading file: ../Preprocessed_Datasets/cleaned_claims_data.csv
🧱 Creating table: Claims
✅ Finished inserting: Claims

🎉 All CSV files inserted successfully!
